<a href="https://colab.research.google.com/github/aebongbing/ESAA/blob/main/0405_%ED%95%B8%EC%A6%88%EC%98%A8_8%EA%B0%95_%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8강. 차원축소

## 8.1 차원의 저주: 훈련 샘플이 각각 수백만 개의 특성을 가지고 있어 훈련을 느리게 하고 좋은 솔루션을 찾기 어렵게 함

고차원은 많은 공간을 가지고 있기 때문에 대부분의 훈련 데이터가 서로 멀리 떨어져 있고 새로운 샘플도 훈련 샘플과 멀리 떨어져 있을 가능성이 높음-> 예측을 하기 위해 훨씬 많은 외삽을 해야함

: 훈련세트의 차원이 클수록 과대적합 위험이 커짐

이론적 해결법: 훈련 샘플의 밀도가 충분히 높아질 때까지 훈련 세트의 크기를 키우기

but 필요한 훈련 샘플의 수는 차원 수가 커짐에 따라 기하급수적으로 늘어남

## 8.2 차원 축소를 위한 접근 방법

### 8.2.1 투영

3차원 공간에서 모든 훈련 샘플이 거의 평면 형태로 놓여져 있음=고차원 공간에 있는 저차원 부분 공간

-> 모든 훈련 샘플을 이 부분 공간에 수직으로 (즉, 샘플과 평면 사이의 가장 짧은 직선을 따라) 투영하면 2D 데이터 샛을 얻게 됨

-> 데이터셋의 차원을 3D에서 2D로 줄임

-> 그러나 스위스 롤 데이터셋처럼 부분 공간이 뒤틀리거나 휘어있기도 하다

-> 우리가 원하는 것은 스위스 롤의 층이 서로 뭉게지지 않고 스위스 롤을 펼쳐 2D 데이터셋을 얻는 것

### 8.2.2 매니폴드 학습

스위스 롤 = 2D 매니폴드의 한 예 =고차원 공간에서 휘어지거나 뒤틀린 2D 모양

d차원 매니폴드는 국부적으로 d차원 초평면으로 보일 수 있는 n차원 공간의 일부
(스위스 롤의 경우, d=2이고 n=3)

매니폴드 학습: 많은 차원 축소 알고리즘이 훈련 샘플이 놓여 있는 매니폴드를 모델링하는 식으로 작동

매니폴드 가정/ 매니폴드 가설: 대부분 실제 고차원 데이터셋이 더 낮은 저차원 매니폴드에 가깝게 놓여있다는 가정

모델을 훈련시키기 전에 훈련 세트의 차원을 감소시키면 훈련 속도는 빨라지지만 항상 더 낫거나 간단한 솔루션이 되는 것은 아님

## 8.3 PCA

PCA=principal component analysis=주성분 분석

가장 인기 있는 차원 축소 알고리즘

데이터에 가장 가까운 초평면을 정의한 다음, 데이터를 이 평면에 투영시킴

### 8.3.1 분산 보존

다른 방향으로 투영하는 것보다 분산이 최대로 보존되는 축을 선택하는 것이 정보가 가장 적게 손실되므로 합리적
= 원본 데이터 셋과 투영된 것 사이의 평균 제곱 거리를 최소화하는 축

### 8.3.2 주성분

고차원 데이터셋이라면 pca는 이전의 두 축에 직교하는 세 번째 축을 찾으며 데이터셋에 있는 차원의 수만큼 네 번째, 다섯 번째,...n번째 축을 찾는다

-> i번째 축을 이 데이터의 i 번째 주성분이라 부름

훈련 세트의 주성분 찾는 법? 특잇값 분해라는 표준 행렬 분해 기술이 있어서 훈련 세트 행렬 X를 세 개 행렬의 행렬 곱셈으로 분해 할 수 있음

### 8.3.3 d차원으로 투영하기

주성분을 모두 추출해냈다면 처음 d개의 주성분으로 정의한 초평면에 투영하여 데이터셋의 차원을 d차원으로 축소시킬 수 있음 이 초평면은 분산을 가능한 한 최대로 보존하는 투영임을 보장함

### 8.3.4 사이킷런 사용하기

SVD 분해 방법을 사용하여 구현하기


In [16]:
from sklearn.datasets import fetch_openml
import numpy as np

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [17]:
from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
#PCA모델을 사용해 데이터셋의 차원을 2로 줄이는 코드
from sklearn.decomposition import PCA

pca=PCA(n_components=2)
X2D=pca.fit_transform(X)

### 8.3.5 설명된 분산의 비율

설명된 분산의 비율(=explained_variance_ratio변수에 저장됨)

In [14]:
pca.explained_variance_

array([334294.06141601, 245433.42703664])

### 8.3.6 적절한 차원 수 선택하기

축소할 차원 수를 임의로 정하기보다는 충분한 분산이 될 때까지 더해야 할 차원 수를 선택하는 것이 간단함

In [18]:
#차원을 축소하지 않고 PCA를 계산한 뒤 훈련 세트의 분산을 95%로 유지하는 데에 필요한 최소 차원의 수 계산
pca=PCA()
pca.fit(X_train)
cumsum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(cumsum >= 0.95)+1

In [19]:
#n_components=d로 설정하여 PCA를 다시 실행
pca=PCA(n_components=0.95)
X_reduced=pca.fit_transform(X_train)

또 다른 방법은 설명된 분산을 차원 수에 대한 함수로 그리는 것

일반적으로 이 그래프에는 설명된 분산의 빠른 성장이 멈추는 변곡점이 있음

여기서는 차원을 약 100으로 축소해도 설명된 분산을 크게 손해보지 않음

### 압축을 위한 PCA

차원을 축소하고 난 후에는 훈련 세트의 크기가 줄어듭니다. 압축된 데이터셋에 PCA 투영의 변환을 반대로 적용하여 차원을 돌릴 수도 있습니다. 원본 데이터와 매우 비슷한 재구성된 데이터와 원본데이터 사이의 평균 제곱 거리를 재구성 오차라고 합니다.

In [20]:
#inverse_transform() 메서드를 사용하여 차원 복원하기
pca=PCA(n_components=154)
X_reduced=pca.fit_transform(X_train)
X_recovered=pca.inverse_transform(X_reduced)

### 8.3.8 랜덤 PCA

svd_solver 매개변수를 'randomized'로 지정하면 사이킷런은 랜덤 PCA라 부르는 확률적 알고리즘을 사용해 처음 d개의 주성분에 대한 근삿값을 빠르게 찾음 d가 n보다 많이 작으면 완전 svd보다 훨씬 빠르다

In [23]:
rnd_pca=PCA(n_components=154, svd_solver='randomized')
X_reduced=rnd_pca.fit_transform(X_train)

### 8.3.9 점진적 PCA

pca구현의 문제는 svd 알고리즘을 실행하기 위해 전체 훈련 세트를 메모리에 올려야 한다는 것입니다. 다행히 점진적 pca 알고리즘이 개발되었습니다 훈련세트를 미니배치로 나눈 뒤 IPCA 알고리즘에 한 번에 하니씩 주입합니다. 이런 방식으로 훈련 세트가 클 때 유용하고 온라인으로 (즉, 새로운 데이터가 준비되는 대로 실시간으로) PCA를 적용할 수도 있다



In [24]:
from sklearn.decomposition import IncrementalPCA

n_batches=100
inc_pca=IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
  inc_pca.partial_fit(X_batch)
X_reduced=inc_pca.transform(X_train)

또 다른 방법은 넘파이의 memmap 파이썬 클래스를 사용해 하드 디스크의 이진 파일에 저장된 매우 큰 배열을 메모리에 들어 있는 것처럼 다루는 것입니다. 이 파이썬 클래스는 필요할 때 데이터를 메모리에 적재합니다. IncrementalPCA는 특정 순간에 배열의 일부만 사용하기 때문에 메모리 부족 문제를 해결할 수 있습니다

In [26]:
filename = "my_mnist.data"
m, n = X_train.shape

X_mm = np.memmap(filename, dtype='float32', mode='write', shape=(m, n))
X_mm[:] = X_train

In [27]:
del X_mm

In [30]:
X_mm=np.memmap(filename, dtype='float32',shape=(m,n))

batch_size=m//n_batches
inc_pca=IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

IncrementalPCA(batch_size=525, n_components=154)